# Dados Textuais — Diretriz Brasileira de Insuficiência Cardíaca

Extrai conteúdo científico da **Diretriz Brasileira de Insuficiência Cardíaca Crônica e Aguda** (SBC), focando nas páginas com conteúdo clínico relevante, removendo seções administrativas e referências.

**Estratégia de extração:**
- Páginas 8-25: Conteúdo científico principal
- Análise de termos cardiológicos frequentes

In [1]:
# Instalar dependências (Colab/Jupyter)
%pip -q install requests pdfplumber
print('Dependências OK')

Note: you may need to restart the kernel to use updated packages.
Dependências OK


In [2]:
import requests
import pdfplumber
from pathlib import Path

url = 'http://publicacoes.cardiol.br/portal/abc/portugues/2018/v11103/pdf/11103021.pdf'

work = Path('.')  # Define 'work' as the current directory
pdf_response = requests.get(url)
pdf_file_path = work / 'downloaded_pdf.pdf'
pdf_file_path.write_bytes(pdf_response.content)
print(f'PDF salvo em: {pdf_file_path}')

PDF salvo em: downloaded_pdf.pdf


In [8]:
with pdfplumber.open(pdf_file_path) as pdf:
    total_pages = len(pdf.pages)
    print(f'📄 Diretriz Brasileira de IC: {total_pages} páginas')
    print('Amostra dos dados extaídos\n')
    
    start_page = 7  # Página 8
    end_page = total_pages
    secoes_extraidas = []

    for i in range(start_page, end_page):
        texto = pdf.pages[i].extract_text()
        if texto and len(texto.strip()) > 100:
            texto_limpo = texto.replace('Diretriz Brasileira de Insuficiência Cardíaca Crônica e Aguda', '')
            texto_limpo = texto_limpo.replace('Diretrizes', '')
            
            secoes_extraidas.append({
                'pagina': i + 1,
                'texto': texto_limpo.strip(),
                'tamanho': len(texto_limpo.strip())
            })
    
    print(f'✅ {len(secoes_extraidas)} páginas com conteúdo científico extraído\n')
    
    for idx, secao in enumerate(secoes_extraidas[2:4]):
        print(f'--- PÁGINA {secao["pagina"]} ({secao["tamanho"]} chars) ---\nResumo:')
        print(secao['texto'][:700])
        print('...\n' + '='*30 + '\n')
    
    texto_completo = '\n\n--- NOVA PÁGINA ---\n\n'.join([s['texto'] for s in secoes_extraidas])


📄 Diretriz Brasileira de IC: 104 páginas

Amostra dos dados extaídos
✅ 97 páginas com conteúdo científico extraído

--- PÁGINA 10 (4379 chars) ---
Resumo:
Quadro 2.1 – Sinais e sintomas de insuficiência cardíaca
Sintomas típicos Sinais mais específicos
Falta de ar/dispneia Pressão venosa jugular elevada
Ortopneia Refluxo hepatojugular
Dispneia paroxística noturna Terceira bulha cardíaca
Fadiga/cansaço Impulso apical desviado para esquerda
Intolerância ao exercício
Sintomas menos típicos Sinais menos específicos
Tosse noturna Crepitações pulmonares
Ganho de peso Taquicardia
Dor abdominal Hepatomegalia e ascite
Perda de apetite e perda de peso Extremidades frias
Noctúria e oligúria Edema periférico
2.1.1. Ecocardiograma natriuréticos auxilia de forma considerável na definição do
O ecocardiograma transtorácico é exame de imagem de diagnóstico de 
...

--- PÁGINA 11 (2493 chars) ---
Resumo:
Tabela 2.1 – Pontos de corte para diagnóstico de insuficiência cardíaca (IC)
Biomarcador IC improváv

In [9]:
import re
from collections import Counter

print('🔬 ANÁLISE DOS DADOS TEXTUAIS EXTRAÍDOS\n')

palavras = texto_completo.lower().split()
print(f'📝 Total de palavras: {len(palavras):,}')
print(f'📝 Palavras únicas: {len(set(palavras)):,}')

termos_cardio = [
    'cardíaca', 'cardíaco', 'coração', 'insuficiência', 'fração', 'ejeção',
    'sistólica', 'diastólica', 'miocárdio', 'ventrículo', 'átrio',
    'tratamento', 'terapia', 'medicamento', 'paciente', 'diagnóstico'
]

print('\n🫀 TERMOS CARDIOLÓGICOS MAIS FREQUENTES:')
for termo in termos_cardio:
    freq = texto_completo.lower().count(termo)
    if freq > 0:
        print(f'• {termo}: {freq} ocorrências')

secoes_identificadas = []
keywords_secoes = {
    'Definição': ['definição', 'conceito', 'caracteriza'],
    'Epidemiologia': ['epidemiologia', 'prevalência', 'incidência'],
    'Diagnóstico': ['diagnóstico', 'diagnóstica', 'exame'],
    'Tratamento': ['tratamento', 'terapêutica', 'medicação'],
    'Prognóstico': ['prognóstico', 'sobrevida', 'mortalidade']
}

print('\n📋 SEÇÕES IDENTIFICADAS NO TEXTO:')
for secao, keywords in keywords_secoes.items():
    encontrado = any(kw in texto_completo.lower() for kw in keywords)
    status = '✅' if encontrado else '❌'
    print(f'{status} {secao}')

arquivo_saida = 'dados_textuais_ic.txt'
with open(arquivo_saida, 'w', encoding='utf-8') as f:
    f.write('=== DIRETRIZ BRASILEIRA DE INSUFICIÊNCIA CARDÍACA ===\n')
    f.write(f'Páginas extraídas: {len(secoes_extraidas)}\n')
    f.write(f'Total caracteres: {len(texto_completo):,}\n')
    f.write(f'Data extração: {__import__("datetime").datetime.now()}\n\n')
    f.write(texto_completo)

print(f'\nDados salvos em: {arquivo_saida}')
print('Extração de dados textuais concluída!')

🔬 ANÁLISE DOS DADOS TEXTUAIS EXTRAÍDOS

📝 Total de palavras: 78,854
📝 Palavras únicas: 17,038

🫀 TERMOS CARDIOLÓGICOS MAIS FREQUENTES:
• cardíaca: 177 ocorrências
• cardíaco: 96 ocorrências
• coração: 5 ocorrências
• insuficiência: 133 ocorrências
• fração: 56 ocorrências
• ejeção: 52 ocorrências
• sistólica: 46 ocorrências
• diastólica: 18 ocorrências
• miocárdio: 32 ocorrências
• ventrículo: 40 ocorrências
• átrio: 5 ocorrências
• tratamento: 221 ocorrências
• terapia: 105 ocorrências
• medicamento: 33 ocorrências
• paciente: 758 ocorrências
• diagnóstico: 97 ocorrências

📋 SEÇÕES IDENTIFICADAS NO TEXTO:
✅ Definição
✅ Epidemiologia
✅ Diagnóstico
✅ Tratamento
✅ Prognóstico

Dados salvos em: dados_textuais_ic.txt
Extração de dados textuais concluída!
